In [1]:
# auto load changes
%load_ext autoreload
%autoreload 2
from pipeline import SFTPipeline

/Users/jr818/miniconda3/envs/myshell/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
sft = SFTPipeline()

In [3]:
sft.build_tokenizer()

In [4]:
sft.build_model(use_lora=False)

In [7]:
sft.predict("Name five different types of fictional characters.")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'\n\n"The first thing you need to do is to use the word "real" in the first place. You can use the words "real," "real." You can also use the "real-world" in your own words.\n"I\'m not going to use this word in the future, but I\'m going to be using it in the next few years," said the author. "I\'m going not to use it in my future, and I\'m not gonna use it'